In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  #显示中文
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import pylab
pylab.rcParams['figure.figsize'] = (15.0, 8.0) # 显示大小

In [36]:
def hand_col(df,col ="发动机转速",n=240):
    
    ####计算col停顿时间
    df["last_notna_time"] = df[df[col].notna()].Time   ##当时不是na的时间
    df_notna_time = df[df[col].notna()].Time
    df_notna_time_diff = (df_notna_time - df_notna_time.shift(1)).dt.total_seconds()
    df["last_notna_time_diff"] = df_notna_time_diff
    df["last_notna_time_diff"] = df["last_notna_time_diff"].fillna(method = "bfill")
    
    df.loc[df["last_notna_time_diff"]> n,col] = df.loc[df["last_notna_time_diff"]> n,col].fillna(0)  #长时间na的填充0
    df[col] = df[col].fillna(method = "ffill")
    return df

In [37]:
def cz(df,idx,col="累计油耗"):
    num = len(idx)
    for i in range(num-1):
        start = idx.iloc[i]
        end = idx.iloc[i+1]
        cz_num = end-start
        start_num = df.loc[start,col]
        end_num = df.loc[end,col]
        fillna = np.linspace(start_num,end_num,cz_num+1)
        df.loc[start:end,col] = fillna
    return df

In [38]:
def hand_col_cz(df,col ="累计油耗"):   ###插值处理
    
    ####计算col停顿间隔
    df["last_notna_idx"] = df[df[col].notna()].idx   ##当时不是na的idx
    df_notna_idx = df[df[col].notna()].idx
    df_notna_idx_diff = (df_notna_idx - df_notna_idx.shift(1))
    df["last_notna_idx_diff"] = df_notna_idx_diff
    df["last_notna_idx_diff"] = df["last_notna_idx_diff"].fillna(method = "bfill")
    
    df.loc[df["last_notna_time_diff"]> n,col] = df.loc[df["last_notna_time_diff"]> n,col].fillna(0)  #长时间na的填充0
    df[col] = df[col].fillna(method = "ffill")
    return df

In [39]:
def hand(df):
#     df = df.drop(["GPS纬度","GPS经度","GPS定位时间"],axis=1)#去除无用数据
    df["Time"] = pd.to_datetime(df["time"].str[:-3])#时间转化,直接去掉小数部分
    
    ####停机以发动机数据为准,处理缺失发动机转数
    df = hand_col(df,col ="发动机转速",n=240)
    
    df = hand_col(df,col ="系统压力",n=240)
    
    df = hand_col(df,col ="转向次数",n=240)
    
    df = hand_col(df,col ="泵送排量",n=240)
    
    df = hand_col(df,col ="泵车状态",n=120)
    
    ####油耗单独填充,插值
    df_notna_idx = df[df["累计油耗"].notna()].idx
    df = cz(df,df_notna_idx,col="累计油耗")
    ###
    
    
    ####泵送计时填充
    df["累计时间"] = df["累计时间"].fillna(method = "ffill")
    
    
    
    ####液压油温用插值
    df_notna_idx = df[df["液压油温"].notna()].idx
    df = cz(df,df_notna_idx,col="液压油温")
    return df

In [40]:
def hand_col_new(df,col ="发动机转速",n=240):
    
    ####计算col停顿时间
    df["last_notna_time"] = df[df[col].notna()].second_idx   ##当时不是na的时间
    df_notna_time = df[df[col].notna()].second_idx
    df_notna_time_diff = (df_notna_time - df_notna_time.shift(1))
    df["last_notna_time_diff"] = df_notna_time_diff
    df["last_notna_time_diff"] = df["last_notna_time_diff"].fillna(method = "bfill")
    
    df.loc[df["last_notna_time_diff"]> n,col] = df.loc[df["last_notna_time_diff"]> n,col].fillna(0)  #长时间na的填充0
    df[col] = df[col].fillna(method = "ffill")
#     df = df.drop(["last_notna_time","last_notna_time_diff"],axis=1)
    return df

In [41]:
df = pd.read_csv("./data/19BC54492983.csv",index_col=0)
df.columns = ["泵车状态","EquipCode","累计时间","液压油温","发动机转速","泵送排量","转向次数","系统压力","累计油耗","time"]
df = df.drop(["EquipCode"],axis=1)

df["Time"] = pd.to_datetime(df["time"].str[:-3])#时间转化,直接去掉小数部分
df["idx"] = np.arange(len(df))
df = hand(df)
df = df.dropna()
df["second"] = df.Time.apply(lambda x: time.mktime(x.timetuple()))
second_start = df.iloc[0].second
df["five_second"] = (df.second - second_start)  // 5
df_mean = df.drop("time",axis=1).groupby("five_second").mean()
df_mean["泵车状态"] = df[["泵车状态","five_second"]].groupby("five_second").max()
start = df.iloc[0].second
end = df.iloc[-1].second
second_index = np.arange(start,end,5)
second_idx = pd.DataFrame({"second_idx":second_index})
new_df = second_idx.join(df_mean)
new_df = new_df.drop(["idx","last_notna_time_diff","second"],axis=1)
new_df["time"] = pd.to_datetime(new_df['second_idx'].values, unit='s',utc=True).tz_convert("Asia/Shanghai").tz_localize(None)

new_df = hand_col_new(new_df,col ="发动机转速",n=240)
new_df = hand_col_new(new_df,col ="系统压力",n=240)
new_df = hand_col_new(new_df,col ="转向次数",n=240)
new_df = hand_col_new(new_df,col ="泵送排量",n=240)
new_df = hand_col_new(new_df,col ="泵车状态",n=240)

new_df["累计油耗"] = new_df["累计油耗"].fillna(method = "bfill")
new_df["累计时间"] = new_df["累计时间"].fillna(method = "bfill")
new_df = new_df.reset_index()
df_notna_idx = new_df[new_df["液压油温"].notna()]["index"]
new_df = cz(new_df,df_notna_idx,col="液压油温")


In [42]:
new_df 

,index,second_idx,泵车状态,累计时间,液压油温,发动机转速,泵送排量,转向次数,系统压力,累计油耗,time,last_notna_time,last_notna_time_diff
0,0,1.563197e+09,0.0,0.0,49.0,562.333333,4.0,0.0,0.0,4.836391,2019-07-15 21:26:08,1.563197e+09,5.0
1,1,1.563197e+09,14.0,0.0,49.0,585.750000,4.0,0.0,0.0,5.032110,2019-07-15 21:26:13,1.563197e+09,5.0
2,2,1.563197e+09,14.0,0.0,49.0,895.200000,4.0,0.0,0.0,5.073394,2019-07-15 21:26:18,1.563197e+09,5.0
3,3,1.563197e+09,14.0,0.0,49.0,1299.000000,4.0,0.0,0.0,5.114679,2019-07-15 21:26:23,1.563197e+09,5.0
4,4,1.563197e+09,14.0,0.0,49.0,1278.600000,4.0,0.0,0.0,5.155963,2019-07-15 21:26:28,1.563197e+09,5.0
5,5,1.563197e+09,14.0,0.0,49.0,1239.600000,4.0,0.0,0.0,5.201835,2019-07-15 21:26:33,1.563197e+09,5.0
6,6,1.563197e+09,14.0,0.0,49.0,1220.500000,4.0,0.0,0.0,5.247706,2019-07-15 21:26:38,1.563197e+09,5.0
7,7,1.563197e+09,14.0,0.0,49.2,1208.400000,4.0,0.0,0.0,5.293578,2019-07-15 21:26:43,1.563197e+09,5.0
8,8,1.563197e+09,14.0,0.0,50.0,1207.750000,4.0,0.0,0.0,5.334862,2019-07-15 21:26:48,1.563197e+09,5.0
9,9,1.563197e+09,14.0,0.0,50.0,1200.400000,4.0,0.0,0.0,5.376147,2019-07-15 21:26:53,1.563197e+09,5.0


In [26]:
ctime = new_df.loc[6]["time"]

In [43]:
window = 12*35
step= window/3
start = 0
lenth = len(new_df)
while start + window < lenth:
    slid = new_df.loc[start:start+window]
    start += step
    time = new_df.loc[start]["time"]
    time = str(time)
    time = time.replace(":","_")
    slid.to_csv("./tmp/sample_"+time+".csv",index=None)